In [1]:
# from selenium import webdriver
# from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import os
import folium
from folium.plugins import MarkerCluster
import pytz

In [9]:
def traj(path, tz):
    
    dfs = pd.read_html(url)
    
    lat = dfs[0].LatitudeLat.iloc[5:]
    lon = dfs[0].LongitudeLon.iloc[5:]
    lat = lat[lat.str.contains(r'^[-0-9]')]
    lon = lon[lon.str.contains(r'^[-0-9]')]
    lat = np.array(['.'.join([x for x in a.split('.')[:2]]) for a in lat])
    lon = np.array(['.'.join([x for x in a.split('.')[:2]]) for a in lon])
    
    lat2 = []
    lon2 = []
    for x in lat:
        if len(x.split('-')) > 1:
            x = x.split('-')[1]
            x = -float(x)
            lat2.append(x)
        else:
            x = float(x)
            lat2.append(x)
    
    for x in lon:
        if len(x.split('-')) > 1:
            x = x.split('-')[1]
            x = -float(x)
            lon2.append(x)
        else:
            x = float(x)
            lon2.append(x)
    
   
    coords = pd.DataFrame({'lat': lat2, 'lon': lon2})
    
    time = dfs[0]['Time (EDT)EDT'].iloc[5:]
    time = time[time.str.contains(r'M$')]
    
    time2 = []
    for x in time:
        time2.append(x.split(' ')[1])
    
    time2 = pd.to_datetime(time2)
    time2 = time2.tz_localize(tz='US/Eastern')
    time2 = time2.tz_convert(tz=tz)
    
    coords['time'] = time2
    
  
    m = visualize(coords)
    return m

In [10]:
def visualize(coords):
    # center to the mean of all points
    m = folium.Map(location=coords[["lat", "lon"]].mean().to_list(), zoom_start=5)

    # if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster
    # marker_cluster = MarkerCluster().add_to(m)

    # draw the markers and assign popup and hover texts
    # add the markers the the cluster layers so that they are automatically clustered
    for i,r in coords.iterrows():
        location = (r["lat"], r["lon"])
        folium.Circle(location=location, 
                     popup = r['time'],
                     tooltip=r['time'],
                     radius=4,
                     color='red')\
        .add_to(m)

    # display the map
    return m

In [11]:
url = 'https://flightaware.com/live/flight/KLM1573/history/20210715/0635Z/EHAM/LGAV/tracklog'
tz = 'Europe/Amsterdam'

In [12]:
m = traj(url, tz)
m